In [17]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio
"""
YOU WILL HAVE TO CHANGE THE FILE PATH TO WORK ON THE FILE SO ON EACH COMPUTER, AS THIS IS TO FOR MY COMPUTER.

"""


# Define the file path
file_path = '/Users/jelletop/Desktop/TIL MSC/YEAR 2/TIL Python/Project/dataset_full.csv'

# Read the CSV file
df_raw = pd.read_csv(file_path, delimiter=";")

"""
Leaving some information for the rest of the project:
The data is structured so that:

Technically you can use the Periods with their code as key: 2015MM01 and 
use the Month/Year columns for the output parts and visualisation. on the X-axis for example. 

another thing for the data
    -> Each quarter already has sum over the values for the months during the quarter 
        -> i.e Q1 values = Sum(Januar,feb,March)
    -> same goes for the years, as they have summed over the quarters or months (which gives the same results)

if you want to do some quarterly values

On the other hand, you have for example:
    -> Total passengers = sheduled + Non_sheduled
        -> As well as Total passengers = Arriving + Departuring 
    -> Asia = West_Asia + East_asia + North_Asia + South_Asia

So you have a lot of breakdown and you can simply pick the columns you want to compare. 

NEWS = North + East + West + South (funny fact newspaper can be interpreted as north/east/west/south/past/and/present/event report )

For europe you only have EU and Non-Eu countries data. Why noth broken into regions idk. 
(Let's just say Europe is one big family in small island in the report, just how we can treat US as a whole)
"""

def preprocessing(df):

    #Filter data, so that we work only with Passengers and only for years above 2015(excluding) 2020
    df = df.iloc[:, :43]
    df = df[ (df['Periods'].str[:4].astype(int) >= 2015)] #add only dates post 2015

    # Function to extract year, month, and quarter from the Periods column
    def extract_date(period):
        #the function returns year, Month, Quarter and since we 
        #have eiither Month or Quarter, the missing one is set as None

        year = period[:4]               # Extract the year
        if 'MM' in period:
            month = str(period[6:])     # Extract month number for MM format
            return year, month, None    
        elif 'KW' in period:
            quarter = str(period[6:])   # Extract quarter number for KW format
            return year, None, quarter  
        return year, None, None  

    # Create new columns for Year, Month, and Quarter
    df[['Year', 'Month_Num', 'Quarter_Num']] = df['Periods'].apply(extract_date).apply(pd.Series)

    # Map months to month names
    month_names = {
        '01': 'January', '02': 'February', '03': 'March', '04': 'April', 
        '05': 'May', '06': 'June', '07': 'July', '08': 'August', 
        '09': 'September', '10': 'October', '11': 'November', '12': 'December'
    }

    # Map quarters to quarter names
    quarter_names = {
        '01': 'Q1', '02': 'Q2', '03': 'Q3', '04': 'Q4'
    }

    #Map airports to their airport quarter  
    airport_names = {
        'A043590':'Amsterdam Schiphol', 'A043596':'Rotterdam','A043591':'Eindhoven'
    }

    # Further data processing 
    df['Month'] = df['Month_Num'].map(month_names)
    df['Quarter'] = df['Quarter_Num'].map(quarter_names)
    df['Airport'] = df['Airports'].map(airport_names)

    # fill in the missing data in the column "month" with the Quarter or year 
    df['Month'] = df['Month'].fillna(df['Quarter'])             
    df['Month'] = df['Month'].fillna(df['Year']) 

    df = df.drop('Quarter', axis=1)         #Quarter is no longer needed 
    # Drop the Month_Num and Quarter_Num columns as they are not needed anymore 
    df.drop(columns=['Month_Num', 'Quarter_Num'], inplace=True)
    df.drop(columns=['Airports'], inplace=True)

    #Local flights are not revelant for this project 
    df.drop(columns=['LocalFlights_2'], inplace=True)

    # List of columns to move and the column after which to insert them
    cols_to_move = ['Month', 'Year', 'Airport']
    insert_after = 'Periods'

    # Get the current columns and find the index of 'Period'
    cols = list(df.columns)
    insert_at = cols.index(insert_after) + 1

    # Remove the columns to move from their original positions
    for col in cols_to_move:
        cols.remove(col)

    # Insert the columns after 'Period'
    for i, col in enumerate(cols_to_move):
        cols.insert(insert_at + i, col)

    # Reorder the DataFrame
    df = df[cols]

    return df

df_processesd = preprocessing(df_raw)

# Display the updated DataFrame
print(df_processesd.head(20))
print(df_processesd.count())



      ID   Periods      Month  Year             Airport  \
274  711  2015MM01    January  2015  Amsterdam Schiphol   
275  712  2015MM02   February  2015  Amsterdam Schiphol   
276  713  2015MM03      March  2015  Amsterdam Schiphol   
277  714  2015KW01         Q1  2015  Amsterdam Schiphol   
278  715  2015MM04      April  2015  Amsterdam Schiphol   
279  716  2015MM05        May  2015  Amsterdam Schiphol   
280  717  2015MM06       June  2015  Amsterdam Schiphol   
281  718  2015KW02         Q2  2015  Amsterdam Schiphol   
282  719  2015MM07       July  2015  Amsterdam Schiphol   
283  720  2015MM08     August  2015  Amsterdam Schiphol   
284  721  2015MM09  September  2015  Amsterdam Schiphol   
285  722  2015KW03         Q3  2015  Amsterdam Schiphol   
286  723  2015MM10    October  2015  Amsterdam Schiphol   
287  724  2015MM11   November  2015  Amsterdam Schiphol   
288  725  2015MM12   December  2015  Amsterdam Schiphol   
289  726  2015KW04         Q4  2015  Amsterdam Schiphol 

In [18]:
# VISUALIZE PASSENGER NUMBER TRENDS INTERNATIONAL VS INTERCONTINENTAL PRE/POST COVID - ALSO TOTAL PASSENGER NUMBERS ARE INCLUDED

def difference_passenger_numbers(df):
    airports = ["Amsterdam Schiphol"]  # Filter on only Amsterdam Schiphol
    df_filtered = df.loc[
        (df["Airport"].isin(airports)) & 
        (df["Periods"].str.contains(r"\d{4}JJ00")) & 
        (df["Year"].astype(int).isin([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]))
    ].copy()  # Filter rows

    # Convert relevant columns to numeric, filling NaNs
    df_filtered["TotalCountryOfOriginDestination_21"] = pd.to_numeric(df_filtered["TotalCountryOfOriginDestination_21"], errors="coerce").fillna(0)
    df_filtered["EuropeTotal_22"] = pd.to_numeric(df_filtered["EuropeTotal_22"], errors="coerce").fillna(0)
    df_filtered["TotalIntercontinentalPassengers_25"] = pd.to_numeric(df_filtered["TotalIntercontinentalPassengers_25"], errors="coerce").fillna(0)
    
    # Group data by Year and sum relevant columns
    df_grouped = df_filtered.groupby("Year")[
        ["TotalCountryOfOriginDestination_21", "EuropeTotal_22", "TotalIntercontinentalPassengers_25"]
    ].sum().reset_index()

    # Rename columns to reflect the legend changes
    df_grouped = df_grouped.rename(columns={
        "TotalCountryOfOriginDestination_21": "Total passengers",
        "EuropeTotal_22": "European passengers",
        "TotalIntercontinentalPassengers_25": "Intercontinental passengers"
    })

    # Create a grouped bar chart to show all three categories separately
    fig = px.bar(df_grouped, 
                 x="Year", 
                 y=["Total passengers", "European passengers", "Intercontinental passengers"], 
                 color_discrete_sequence=["#636EFA", "#EF553B", "#00CC96"],
                 labels={"value": "Number of Passengers", "Year": "Year"}, 
                 title="Yearly Total Passenger Numbers for Amsterdam Schiphol",
                 barmode="group",  # Group the bars
                 height=600)

    # Customize hover data to show totals and individual segments
    fig.update_traces(hovertemplate='%{x}: %{y}')
    fig.update_layout(barmode='group')

    fig.show()

# Assuming df_processed is already loaded
difference_passenger_numbers(df_processesd)


In [19]:
# VISUALIZE PASSENGER NUMBER TRENDS INTERNATIONAL VS INTERCONTINENTAL PRE/POST COVID

def create_line_graph(df):
    airports = ["Amsterdam Schiphol"]  # Filter on only Amsterdam Schiphol
    df_filtered = df.loc[
        (df["Airport"].isin(airports)) & 
        (df["Periods"].str.contains(r"\d{4}JJ00")) & 
        (df["Year"].astype(int).isin([2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]))
    ].copy()  # Filter rows

    # Convert columns to numeric, filling NaNs
    df_filtered["EuropeTotal_22"] = pd.to_numeric(df_filtered["EuropeTotal_22"], errors="coerce").fillna(0)
    df_filtered["TotalIntercontinentalPassengers_25"] = pd.to_numeric(df_filtered["TotalIntercontinentalPassengers_25"], errors="coerce").fillna(0)

    # Group data by Year and Airport, then sum values
    df_grouped = df_filtered.groupby(["Year"])[
        ["EuropeTotal_22", "TotalIntercontinentalPassengers_25"]
    ].sum().reset_index()

    # Rename columns for better legend labels
    df_grouped = df_grouped.rename(columns={
        "EuropeTotal_22": "European passengers",
        "TotalIntercontinentalPassengers_25": "Intercontinental passengers"
    })

    # Create a line graph comparing European and Intercontinental passengers
    fig = px.line(df_grouped, 
                  x="Year", 
                  y=["European passengers", "Intercontinental passengers"],
                  labels={"value": "Number of Passengers", "Year": "Year"},
                  title="Intercontinental vs European Passenger Numbers for Amsterdam Schiphol",
                  markers=True)  # Add markers for each point

    # Customize layout and trace appearance
    fig.update_traces(mode='lines+markers')
    fig.update_layout(legend_title_text='Passenger Types')

    fig.show()

# Assuming df_processed is already loaded
create_line_graph(df_processesd)
